# Day 3: Crossed Wires

In [1]:
import numpy

f = open('input.txt', 'r')
wires = [line.split(',') for line in f.readlines()]

## Part 1

The gravity assist was successful, and you're well on your way to the Venus refuelling station. During the rush back on Earth, the fuel management system wasn't completely installed, so that's next on the priority list.

Opening the front panel reveals a jumble of wires. Specifically, two wires are connected to a central port and extend outward on a grid. You trace the path each wire takes as it leaves the central port, one wire per line of text (your puzzle input).

The wires twist and turn, but the two wires occasionally cross paths. To fix the circuit, you need to find the intersection point closest to the central port. Because the wires are on a grid, use the Manhattan distance for this measurement. While the wires do technically cross right at the central port where they both start, this point does not count, nor does a wire count as crossing with itself.

For example, if the first wire's path is R8,U5,L5,D3, then starting from the central port (o), it goes right 8, up 5, left 5, and finally down 3:

```
...........
...........
...........
....+----+.
....|....|.
....|....|.
....|....|.
.........|.
.o-------+.
...........
```

Then, if the second wire's path is U7,R6,D4,L4, it goes up 7, right 6, down 4, and left 4:

```
...........
.+-----+...
.|.....|...
.|..+--X-+.
.|..|..|.|.
.|.-X--+.|.
.|..|....|.
.|.......|.
.o-------+.
...........
```

These wires cross at two locations (marked X), but the lower-left one is closer to the central port: its distance is 3 + 3 = 6.

Here are a few more examples:

- R75,D30,R83,U83,L12,D49,R71,U7,L72
- U62,R66,U55,R34,D71,R55,D58,R83 = distance 159
- R98,U47,R26,D63,R33,U87,L62,D20,R33,U53,R51
- U98,R91,D20,R16,D67,R40,U7,R15,U6,R7 = distance 135

What is the Manhattan distance from the central port to the closest intersection?

In [2]:
# From: https://algorithmtutor.com/Computational-Geometry/Check-if-two-line-segment-intersect/
# checks if line segment p1p2 and p3p4 intersect
def intersect(p1, p2, p3, p4):
    def cross_product(p1, p2):
	    return p1[0] * p2[1] - p2[0] * p1[1]
    # returns the cross product of vector p1p3 and p1p2
    # if p1p3 is clockwise from p1p2 it returns +ve value
    # if p1p3 is anti-clockwise from p1p2 it returns -ve value
    # if p1 p2 and p3 are collinear it returns 0
    def direction(p1, p2, p3):
	    return  cross_product(numpy.subtract(p3, p1), numpy.subtract(p2, p1))
    
    # checks if p lies on the segment p1p2
    def on_segment(p1, p2, p):
        return min(p1[0], p2[0]) <= p[0] <= max(p1[0], p2[0]) and min(p1[1], p2[1]) <= p[1] <= max(p1[1], p2[1])

    d1 = direction(p3, p4, p1)
    d2 = direction(p3, p4, p2)
    d3 = direction(p1, p2, p3)
    d4 = direction(p1, p2, p4)

    if ((d1 > 0 and d2 < 0) or (d1 < 0 and d2 > 0)) and ((d3 > 0 and d4 < 0) or (d3 < 0 and d4 > 0)):
        return True
    elif d1 == 0 and on_segment(p3, p4, p1):
        return True
    elif d2 == 0 and on_segment(p3, p4, p2):
        return True
    elif d3 == 0 and on_segment(p1, p2, p3):
        return True
    elif d4 == 0 and on_segment(p1, p2, p4):
        return True
    else:
        return False

def wireIntersections(p1, p2, p3, p4):
    def line_intersection(line1, line2):
        xdiff = (line1[0][0] - line1[1][0], line2[0][0] - line2[1][0])
        ydiff = (line1[0][1] - line1[1][1], line2[0][1] - line2[1][1])

        def det(a, b):
            return a[0] * b[1] - a[1] * b[0]

        div = det(xdiff, ydiff)
        if div == 0:
           return

        d = (det(*line1), det(*line2))
        x = det(d, xdiff) / div
        y = det(d, ydiff) / div
        return x, y
    
    if intersect(p1, p2, p3, p4):
        return line_intersection((p1, p2), (p3, p4))
    
def followWire(start, directions):
    wire = list()
    wire.append(start)
    
    for jump in directions:
        direction = jump[0]
        distance = int(jump[1:])
        
        if direction == "R":
                wire.append(tuple(map(sum, zip(wire[-1], (distance, 0)))))
        elif direction == "L":
                wire.append(tuple(map(sum, zip(wire[-1], (-1*distance, 0)))))       
        elif direction == "U":
                wire.append(tuple(map(sum, zip(wire[-1], (0, distance)))))
        elif direction == "D":
                wire.append(tuple(map(sum, zip(wire[-1], (0, -1*distance)))))      
        else:
            print("Direction not supported")
            break
        
    return(wire)

wireLocations = list()
for wire in wires:
    wireLocations.append(followWire((0,0), wire))

intersections = list()
for i in range(1, len(wireLocations[0])-1):
    for j in range(1, len(wireLocations[1])-1):
        intersection = wireIntersections(wireLocations[0][i], wireLocations[0][i+1], wireLocations[1][j], wireLocations[1][j+1])
        (intersections.append(intersection) if intersection is not None else None)

distances = list()
for point in intersections:
    distances.append(abs(point[0])+abs(point[1]))
    
print("Solution: ", str(int(min(distances))))

Solution:  1195


## Part 2

It turns out that this circuit is very timing-sensitive; you actually need to minimize the signal delay.

To do this, calculate the number of steps each wire takes to reach each intersection; choose the intersection where the sum of both wires' steps is lowest. If a wire visits a position on the grid multiple times, use the steps value from the first time it visits that position when calculating the total value of a specific intersection.

The number of steps a wire takes is the total number of grid squares the wire has entered to get to that location, including the intersection being considered. Again consider the example from above:

```
...........
.+-----+...
.|.....|...
.|..+--X-+.
.|..|..|.|.
.|.-X--+.|.
.|..|....|.
.|.......|.
.o-------+.
...........
```

In the above example, the intersection closest to the central port is reached after 8+5+5+2 = 20 steps by the first wire and 7+6+4+3 = 20 steps by the second wire for a total of 20+20 = 40 steps.

However, the top-right intersection is better: the first wire takes only 8+5+2 = 15 and the second wire takes only 7+6+2 = 15, a total of 15+15 = 30 steps.

Here are the best steps for the extra examples from above:

- R75,D30,R83,U83,L12,D49,R71,U7,L72
- U62,R66,U55,R34,D71,R55,D58,R83 = 610 steps
- R98,U47,R26,D63,R33,U87,L62,D20,R33,U53,R51
- U98,R91,D20,R16,D67,R40,U7,R15,U6,R7 = 410 steps

What is the fewest combined steps the wires must take to reach an intersection?

In [3]:
def measureDistanceToIntersection(wireLocations):
    distances = list()
    for i in range(1, len(wireLocations[0])-1):
        for j in range(1, len(wireLocations[1])-1):
            intersection = wireIntersections(wireLocations[0][i], wireLocations[0][i+1], wireLocations[1][j], wireLocations[1][j+1])
            if intersection is not None:
                steps = 0
                for l in range(0, i):
                    if i-1 == 0:
                        continue
                    d = numpy.subtract(wireLocations[0][l], wireLocations[0][l+1])
                    steps += abs(d[0]) + abs(d[1])
                d = numpy.subtract(intersection, wireLocations[0][i])
                steps += abs(d[0]) + abs(d[1])
                for m in range(0, j):
                    if j-1 == 0:
                        continue
                    d = numpy.subtract(wireLocations[1][m], wireLocations[1][m+1])
                    steps += abs(d[0]) + abs(d[1])
                d = numpy.subtract(intersection, wireLocations[1][j])
                steps += abs(d[0]) + abs(d[1])
                distances.append(steps)
    return(distances)

print("Solution: " + str(int(min(measureDistanceToIntersection(wireLocations)))))

Solution: 91518


# Test

In [4]:
testWire1 = [line.split(',') for line in ["R8,U5,L5,D3", "U7,R6,D4,L4"]]

wireLocations = list()
for wire in testWire1:
    wireLocations.append(followWire((0,0), wire))
    
intersections = list()
for i in range(len(wireLocations[0])-1):
    for j in range(len(wireLocations[1])-1):
        intersection = wireIntersections(wireLocations[0][i], wireLocations[0][i+1], wireLocations[1][j], wireLocations[1][j+1])
        (intersections.append(intersection) if intersection is not None else None)

distances = list()
for point in intersections:
    distances.append(abs(point[0])+abs(point[1]))

print(testWire1)
print(wireLocations)
print(intersections)
print(distances)

[['R8', 'U5', 'L5', 'D3'], ['U7', 'R6', 'D4', 'L4']]
[[(0, 0), (8, 0), (8, 5), (3, 5), (3, 2)], [(0, 0), (0, 7), (6, 7), (6, 3), (2, 3)]]
[(0.0, 0.0), (6.0, 5.0), (3.0, 3.0)]
[0.0, 11.0, 6.0]


In [5]:
print(measureDistanceToIntersection(wireLocations))

[30.0, 40.0]
